In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

import os
import gc
import time
import multiprocessing

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

import scipy.sparse as sparse
import scipy.stats as stats

# Чтобы pandas отображал все столбцы
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 170)


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Any results you write to the current directory are saved as output.
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [51]:
train = pd.read_csv('/kaggle/input/recommendationsv4/train.csv', low_memory=False)
test = pd.read_csv('/kaggle/input/recommendationsv4/test.csv', low_memory=False)
submission = pd.read_csv('/kaggle/input/recommendationsv4/sample_submission.csv')

# read the meta_Grocery_and_Gourmet_Food.json
# It is an additional products information
with open('/kaggle/input/recommendationsv4/meta_Grocery_and_Gourmet_Food.json') as f:
    meta_list = []
    for line in f.readlines():
        meta_list.append(json.loads(line))
        
meta = pd.DataFrame(meta_list)

In [ ]:
print(train.shape)
print(test.shape)
print(meta.shape)

In [ ]:
def count_empty_values_and_show(data, name=''):
    """
    Отображает в % кол-во пропущенных значений по каждому признаку.
    Графически отобразит пропуски в данных с помощью библиотеку seaborn.
    
    Input:
        data: a DataFrame object,
        name: a string object - name of dataset - optional
    Return: None
    """
    # определяем цвета 
    # желтый - пропущенные данные, синий - не пропущенные
    colours = ['#000099', '#ffff00']
    sns.heatmap(data.isnull(), cmap=sns.color_palette(colours))
    
    print(f"Empty values in {name.upper()}:")
    for col in data.columns:
        pct_missing = np.mean(data[col].isnull())
        print('--{} - {}%'.format(col, round(pct_missing*100, 2)))


## Посмотрим на данные

датасет train

In [ ]:
display(train.head(3))
count_empty_values_and_show(train, name='train')

датасет test

In [ ]:
display(test.head(3))
count_empty_values_and_show(test, name='test')

датасет meta

In [ ]:
display(meta.head(3))
count_empty_values_and_show(meta, name='meta')

**train и test**  
В трейне и тестовой выборке признаки style и image имеют большое кол-во пропусков. Так же признак style, по своему содержанию содержит не интересующую нас информацию. Удалим их. При этом признак vote имеет примерно 86% пропусков. Данный признак показывает кол-во проголосовавших за написанный отзыв. Чем больше значение, тем большему кол-ву пользователей данный отзыв показался значимым или полезным.  При  этом пропуски в данном признаке могут означать не отсутствие данных, а тот факт, что за отзыв никто ни проголосовал и мы можем без ущерба для будущей модели заменить пропуски нулями.  Конечно, стоит протестировать в дальнейшем модель и без этого признака.  


**meta**  
В наборе данных следующие признаки имеют пропуски, а именно:  
- also_view - 58% пропусков - попробуем заполнить пропуски нулями, что будет означать, что пользователь не смотрел больше других товаров.
- price - 54% пропусков - попробуем обучить модель на данных, заполненных средним или медианым значением из текущей категории товара
- also_buy - 71% пропусков - попробуем заполнить пропуски нулями, что будет означать, что пользователь не покупал больше других товаров.
- image - 48% пропусков - удаляем  
- date - признак почти пустой - удаляем  
- feature - признак почти пустой - удаляем  
- similar_item - признак полностью пустой - удаляем  
- tech1 - признак полностью пустой - удаляем  
- fit - признак полностью пустой - удаляем  

- details - информация, необходимая в качестве технической информации для отображения сведений о товаре на web-ресурсе. Для нашей задачи они не пригодятся. Удаляем

In [ ]:
train.drop(columns=['style', 'image'], inplace=True)
test.drop(columns=['style', 'image'], inplace=True)
meta.drop(columns=['image', 'date', 'feature', 'similar_item', 'tech1', 'fit', 'details'], inplace=True)

## Объединение датасетов

### Предобработка meta

In [ ]:
meta.info()

In [ ]:
# получаем ошибку: TypeError: unhashable type: 'list'
# meta.drop_duplicates(inplace=True)

Почистить  meta от дублей не представляется возможным ввиду присутсвия в нем данных типа list, а именно:  
- category - это список категорий, начиная с главной и заканчивая той, в которой находится товар   
- description - это список строк, описывающий товар.  
- also_view - список идентификаторов, а точнее ключей asin, которые так же смотрел пользователь  
- also_buy - список идентификаторов, а точнее ключей asin, которые так же покупал пользователь    

Для простоты обработки с целью удаления дублей, сделаем из списка значений в признаке category преобразуем их в строки, то же самое сделаем с признаком description.  
С признаки also_view и also_buy заменим на кол-во ключей в списке. Таким образом, мы получим не конкретные идентификаторы просмотренных или купленных товаров, а из кол-во.

Для признаков category и description преобразуем список строк в строку с помощью метода join.  
Признаки also_view и also_buy временно заменим на строковое отображение, чтобы избежать конфликтов ввиду проблем обработки встроенными функциями pandas признаков со списками в качестве значений.  
Так же на основе этих признаков, на этапе feature engineering можно создать новые признаки, для каждого конкретного товара. Например:
- кол-во просмотренных товаров конкретным пользователем
- кол-во купленных товаров конкретным пользователем
- подсчитать сколько всего людей смотрели данный товар,  
- сколько всего людей купили данный товар  
- отношение кол-ва покупок к кол-ву просмотров данного товара  
- и т.д.



Признаки description, also_view, also_buy имеют пропуски в данных.  
Для корректного использования метода join, который выбрасывает ошибку, если значение в ячейке отсутсвует, заменим пропуски значением **'nan'**

In [ ]:
# заполнение пропусков временной заглушкой
cols_to_fill = ['description', 'also_view', 'also_buy']
meta[cols_to_fill] = meta[cols_to_fill].fillna(value='nan', axis=1)

#  переведем в строки
meta.category = meta.category.apply(lambda lst: "|".join(lst))
meta.description = meta.description.apply(lambda val: "|".join(val) if val != 'nan' else 'nan')
meta.also_view = meta.also_view.apply(lambda val: "|".join(val) if val != 'nan' else 'nan')
meta.also_buy = meta.also_buy.apply(lambda val: "|".join(val) if val != 'nan' else 'nan')

# Поменяем тип данных признака rank на object
meta['rank'] = meta['rank'].astype('str')


#  Удалим дубли
print(f"Shape before {meta.shape}")
meta.drop_duplicates(inplace=True)
print(f"Shape after {meta.shape}")

In [ ]:
# Добавим маркер тренировочного набора данных
train['is_train'] = 1
test['is_train'] = 0

# Объединим  датасеты и данные из meta по идентификатору asin (Amazon Standard Identification Number)
# train
train.drop_duplicates(inplace=True)
df_new_train = pd.merge(train, meta, on='asin')
# test
test.drop_duplicates(inplace=True)
df_new_test = pd.merge(test, meta, on='asin').sort_values('Id')
    
# объединим данные
df = pd.concat([df_new_train, df_new_test], ignore_index=True)
print(f"train data has shape: {train.shape}")
print(f"test data has shape: {test.shape}")
print(f"Common data has shape: {df.shape}")

Для оптимизации оперативной памяти, удалим уже не нужные данные

In [ ]:
del train
del test
del df_new_train
del df_new_test
del meta

# garbage collect
gc.collect()

Предобработку и объединение датасетов произвели.  
Теперь посмотрим на данные

#### Несколько полезных фукнций

In [ ]:
def diagnostic_plots(df, variable):
    plt.figure(figsize=(15,6))
    # гистограмма
    plt.subplot(1, 2, 1)
    df[variable].hist(bins=50)
    
    ## Q-Q plot
    plt.subplot(1, 2, 2)
    stats.probplot(df[variable], dist="norm", plot=plt)
    plt.suptitle(variable)
    plt.show()
    
# diagnostic_plots(df[df.is_train == 1], 'overall')

def diagnostic_plots_2(df, variable, title):
    fig, ax = plt.subplots(figsize=(10,7))
    # гистограмма
    plt.subplot(2, 2, 1)
    df[variable].hist(bins=30)
    ## Q-Q plot
    plt.subplot(2, 2, 2)
    stats.probplot(df[variable], dist="norm", plot=plt)
    # ящик с усами
    plt.subplot(2, 2, 3)
    sns.violinplot(x=df[variable])    
    # ящик с усами
    plt.subplot(2, 2, 4)
    sns.boxplot(x=df[variable])  
    fig.suptitle(title)
    plt.show()

In [ ]:
#  вывод на экран описательной статистики по конкретному признаку
def describe_cat_feature(data, feature_name):
    print(f'Описательная статистика признака {feature_name}:')
    print(data[feature_name].describe())
    
    print('\nРаспределение признака в %:')
    print(data[feature_name].value_counts(normalize=True))
    sns.countplot(x=feature_name, data=data)

###  Поочередный анализ признаков

In [ ]:
df.info()

#### признак **overall**  
overall - оценка поставленная пользователем за товар по пятибальной шкале

In [ ]:
describe_cat_feature(df[df.is_train == 1], 'overall')

Видно, что большая часть товаров item-ов имеет положительную оценку пользователями.  
Таким образом, если считать, что пользователю товар понравился и он поставил оценку 4 или 5, то мы имеем 84.8% положительных отзывов в тренировочной выборке.

#### признак **verified**  
verified - подтвержден отзыв или нет

In [ ]:
describe_cat_feature(df[df.is_train == 1], 'verified')

In [ ]:
# Посмотрим на статистики по тестовому датасету
describe_cat_feature(df[df.is_train == 0], 'verified')

Распределение признака одинаковое в обоих датасетах.  
86% отзывов верифицированы.  

In [ ]:
# Переведем признак verified к типу int64 с помощью метода astype
df.verified = df.verified.astype('uint8')

#### признак reviewTime  
reviewTime - дата написания отзыва  
Пока не будем его рассматривать, так как в наборе данных имеется более удобный для анализа признак unixReviewTime.  


#### признак unixReviewTime  
unixReviewTime - время в unix формате. 

In [ ]:
tsmin = datetime.utcfromtimestamp(df.unixReviewTime.min()).strftime('%Y-%m-%d %H:%M:%S')
tsmax = datetime.utcfromtimestamp(df.unixReviewTime.max()).strftime('%Y-%m-%d %H:%M:%S')

print(tsmin)
print(tsmax)

Отзывы оставлены в период с Августа 2000 года по Октябрь 2018.  
На этапе feature engineering заменим значения в столбце на категории, а так же создадим новые признаки на основе ReviewTime перед его удалением

#### признак asin  
Это чисто технических признак, который является уникальным идентификатором товара на Амазон.  
В нашем случае мы использовали его для соединения таблиц данных.
На этапе генерации новых признаков он понадобиться для создания новых фичей.  

#### признак reviewerName  
reviewerName - имя пользователя  

In [ ]:
df.reviewerName.value_counts()

Это строковый признак. Самое частотное значение "Amazon Customer". Видимо в форме отзыва требуется указать имя пользователя, но не все хотят этого делать. Поэтому заполняют поля шаблоном.  
Стоит удалить данный признак.

In [ ]:
df.drop(labels='reviewerName', axis=1, inplace=True)

####  признак vote  
vote - кол-во голосов, отданных за конкретный отзыв.

In [ ]:
# посмотрим на значения признака
print(df.loc[~df.vote.isna(), 'vote'].sample(100).to_list())

In [ ]:
describe_cat_feature(df, 'vote')

Самое частотное значение 2, такое гол-во голосов имеет 59308 отзывов.

Предположим, что отсутсвующие значения в данном признаке это не пропуски, а условные нули. То есть за отзыв просто никто не проголосовал.  
Следует заполнить пропуски нулями, но перед этим стоит создать бинарный признак является ли значение реальным или искусственным.

In [ ]:
# булевый признак
df['vote_isNAN'] = pd.isna(df['vote']).astype('uint8')

# заполним пропуски нулями
df.vote = df.vote.fillna(value='0')

# Для корректной смены типа данный, надо значения типа 1,626 или 1,383 избавить от знака запятой.  
df.vote = df.vote.str.replace(',', '')
df.vote = df.vote.astype('int32')

In [ ]:
diagnostic_plots_2(df, 'vote', 'vote - original')

видим большое скопление в околонулевой зоне, что не удивительно, ведь мы сами заполнили пропуски нулями.  
По графику boxplot наблюдаем наличие выбросов.
распределение не похоже на нормальное.

Опытным путем стало ясно, что привести распределение признака к нормальную не представляется возможным.  
Что было испробовано:  
- Логарифмирование  
- Обратное преобразование  
- Квадратный корень  
- Преобразование Бокса-Кокса  
- Преобразование Йео-Джонсона  

Решено сделать из него категориальный признак по кол-ву голосов: низкий, средний, высокий и очень высокий.

In [ ]:
# сделаем категории по количеству голосов за отзыв
df["vote"] = df["vote"].apply(lambda x: 'low' if x < 5 else 'high' if x > 50 else 'middle')

In [ ]:
describe_cat_feature(df, 'vote')

#### признаки userid и itemid  
Идентификаторы пользователей и товаров. 

In [ ]:
print(f"кол-во уникальных пользвателей {df.userid.nunique()}")
print(f"кол-во уникальных товаров {df.itemid.nunique()}")

#### признак rating  
rating - целевой признак

In [ ]:
describe_cat_feature(df[df.is_train == 1], 'rating')

В тренировочном наборе данных целевой признак rating не сбалансирован. Пользователь товары нравятся в 5,6 раз чаще чем не нравятся.

#### признак category

Признак category на этапе соединения датасетов был преобразован из списка строк в строку с маркером для последующего разделения и генерации новых категориальных признаков.

Создадим временный признак из category - кол-во вложенных категорий, чтобы принять решение о создании на основе category отдельных признаков.  

In [ ]:
df['subcat_count'] = df.category.apply(lambda s: len(s.split('|'))).astype('uint8')

In [ ]:
#  Подсчитаемт кол-во каждого варианта
df['subcat_count'].value_counts()

Создадим 2 новых признака на основе category - main_cat, sub_cat, где main_cat - это основная категория, а sub_cat - это первая подкатегория, к которой относится данный товар.  

In [ ]:
df['main_cat'] = df.category.apply(lambda s: s.split('|')[0])
df['sub_cat'] = df.category.apply(lambda s: s.split('|')[1])

In [ ]:
#  Сколько уникальных значений в main_cat
describe_cat_feature(df, 'sub_cat')

In [ ]:
#  посмотрим на гистрограмму sub_cat под другим углом
sns.catplot(y="sub_cat", kind="count",
            palette="pastel", edgecolor=".6",
            data=df)

Из статистик и графиков признака sub_cat видно, что top-3 являются товары из следующих категорий:  
- Beverages  
- Cooking & Baking  
- Snack Foods  

Нет смысла оставлять признак main_cat, так как он не информативен.  
Так же нет смысла создавать дополнительные категориальные признаки, например из последних значений признака category, так как это значительно увеличит матрицу признаков и увеличит требования к вычислительной мощности и памяти.  
Однако стоит ради интереса посмотреть сколько уникальных итоговых категорий мы имеем.  


In [ ]:
df['last_cat'] = df.category.apply(lambda s: s.split('|')[-1])
df.last_cat.nunique()

Уникальных значений last_cat 967.  
Использовать их я не буду для экономии ресурсов.  
Оставим только признак sub_cat, а остальные удалим.

In [ ]:
drop_cols = ['category', 'main_cat', 'subcat_count', 'last_cat']
df.drop(columns=drop_cols, inplace=True)

дополнительная обработка sub_cat

In [ ]:
df.sub_cat = df.sub_cat.str.replace(' & ', '_') \
                       .str.replace(', ', '_') \
                       .str.replace(' ', '_') \
                       .str.lower()

In [ ]:
df.sub_cat.unique()

#### признаки description и title  
title и description - название и описание товара.  
На мой взгляд данные признаки являются достаточно ценными, однако слова из заголовка частично дублируются в описании.  
Поэтому стоит совместить данные признаки в один для дальнейшей обработки.  
Но перед этим посмотрим на рандомные значения данных признаков


признак description  
Признак имеет некоторые объекты со значением 'nan', которое было добавлено специально на этапе предобработки датасета meta.  
В настоящий момент стоит вернуть изначальное значение np.nan для корректной отработки дальнейших скриптов.

In [ ]:
df.description = df.description.apply(lambda s: np.nan if s == 'nan' else s)
df.description.isnull().sum()

In [ ]:
# добавим булевый признак
df['desc_isNAN'] = pd.isna(df['description']).astype('uint8')

In [ ]:
#  заполним пропуски в description значением 'noData'
df['description'].fillna(value='noData', inplace=True)

Визаульный анализ признака показал наличие в тексте чисто технической подстроки следующего содержания:  
"Statements regarding dietary supplements have not been evaluated by the FDA and are not intended to diagnose, treat, cure, or prevent any disease or health condition."  
Удалим эту подстроку и заменим маркер "|" на " ".

In [ ]:
sub_str = 'Statements regarding dietary supplements have not been evaluated by the FDA and are not intended to diagnose, treat, cure, or prevent any disease or health condition.'

#  сначала соединим строки, не содержащие не желательную подстроку
df.loc[~df.description.str.contains(sub_str), 'description'] = df.loc[~df.description.str.contains(sub_str), 'description'].apply(lambda s: ' '.join(s.split('|')))

#  теперь очередь строк содержащих удаляемый шаблон
df.loc[df.description.str.contains(sub_str), 'description'] = df.loc[df.description.str.contains(sub_str), 'description'].apply(lambda s: ' '.join(s.split('|')[:-1]))

признак title

Среди значений встречаются чисто технические записи, которые являются не чем иным как javascript кодом ошибочно записанным в данный признак.  
Стоит удалить не корректные значения и заменить из на np.nan

In [ ]:
#  посмотрим сколько таких записей имеется в нашем наборе данных
sub_str = 'var aPageStart'
#  Подсчитаем кол-во вхождений шаблона
template_count = df[df.title.str.contains(sub_str)].shape[0]
print(f'В title шаблон встречается {template_count} раз.')

# заменим ошибочные значения на пропуски
df.loc[df.title.str.contains(sub_str), 'title'] = np.nan
df.title.isnull().sum()

In [ ]:
# добавим булевый признак
df['title_isNAN'] = pd.isna(df['title']).astype('uint8')

#  заполним пропуски в description значением 'noData'
df['title'].fillna(value='noData', inplace=True)

Теперь оба признака подготовлены для объединения, с целью последующего TF-IDF encoding-га на шаге генерации признаков.  
Назовем признак на основе title и description i_desc

In [ ]:
df['i_desc'] = df.title + ' ' + df.description

In [ ]:
# теперь удалим title и description
drop_cols = ['title', 'description']
df.drop(columns=drop_cols, inplace=True)

#### признак brand

In [ ]:
# кол-во пропусков
df.brand.isnull().sum()

In [ ]:
# заполним пропуски
df.brand = df.brand.fillna(method='backfill')

In [ ]:
# приведем к нижнему регистру
df.brand  = df.brand.str.lower()

In [ ]:
# get top 10 brands
df.brand.value_counts()[:10]

Использовать все уникальные значения признака brand - это перебор.  
Выделим топ 10 значений и присвоим всем остальным значение 'other'.

In [ ]:
top_ten_brands = df.brand.value_counts()[:10].index.to_list()

In [ ]:
df.brand = df.brand.apply(lambda s: s if s in top_ten_brands else 'other')

In [ ]:
describe_cat_feature(df, 'brand')

Признак brand достаточно разнообразен. 
Были оставлены названия 10 самых частотных брендов, остальные заменены на значение 'other'.  
Значения признака будут использованы на этапе генерирования новых фичей на следующих этапах.

#### признак rank

In [ ]:
# вернем пропусках из реальное значение np.nan
df['rank'] = df['rank'].apply(lambda s: np.nan if s == 'nan' else s)

# выведем на экран кол-во пропусков
df['rank'].isnull().sum()

In [ ]:
# Создадим сначала булевый признак является ли значение реальным или искусственным
df['rank_isNAN'] = pd.isna(df['rank']).astype('uint8')

In [ ]:
# визуально оценим несколько образцов
df['rank'].sample(30).to_list()

В данном признаке совмещено значение рейтинга и категории, в которой данных товар присутсвует.  
Сложно сказать по какому принципу составлялся рейтинг, однако можно попробовать использовать эти данные.  
Для этого создадим 2 признака rank_val и rank_cat

In [ ]:
df['rank_val'] = np.nan
df['rank_cat'] = np.nan

def processing_rank(row):
    if pd.isnull(row['rank']):
        return row
    try:
        rank_str = row['rank'].split('(')[0]

        # rank_val
        rank_val = rank_str.split('in')[0]
        rank_val = rank_val.replace('>', '') \
                            .replace('#', '') \
                            .replace(',', '') \
                            .replace('[', '') \
                            .replace("'", '') \
                            .strip()
        row['rank_val'] = rank_val

        # rank_cat
        if ' in ' in rank_str:
            rank_cat = rank_str.split(' in ')[-1].strip()
        else:
            rank_cat = rank_str.split('in')[-1].strip()
        row['rank_cat'] = rank_cat
    
    except Exception as e:
        print(f"[ERROR]:", str(e), '=>', row['rank'], 'type:', type(row['rank']))
        
    return row


df.loc[:, ['rank', 'rank_val', 'rank_cat']] = df.loc[:, ['rank', 'rank_val', 'rank_cat']].apply(processing_rank, axis=1)

In [ ]:
coffe_break()

Посмотрим на статистики 2-х получившихся производных признаков

признак rank_val

In [ ]:
# поменять тип данных на int
df.rank_val = df.rank_val.astype('float64')

In [ ]:
df.rank_val.describe()

In [ ]:
diagnostic_plots_2(df, 'rank_val', 'rank_val - original')

Заполним пропуски медианой


In [ ]:
rank_median = df.rank_val.median()
df.rank_val.fillna(value = rank_median, inplace=True)

In [ ]:
diagnostic_plots_2(df, 'rank_val', 'rank_val - original, after filling empty items')

Опытным путем стало ясно, что привести распределение признака к нормальному лучше всего получилось при использовании преобразования Йео-Джонсона.  
Что было испробовано:  
- Логарифмирование  
- Обратное преобразование  
- Квадратный корень  
- Возведение в степень несколько вариантов
- Преобразование Бокса-Кокса  
- Преобразование Йео-Джонсона

Преобразование Йео-Джонсона

In [ ]:
df['rank_val_yeojohnson'], param = stats.yeojohnson(df['rank_val']) 
print('Оптимальное значение λ = {}'.format(param))
# diagnostic_plots_2(df, 'rank_val_yeojohnson', 'rank_val - yeojohnson')

In [ ]:
drop_cols = ['rank_val', 'rank']
df.drop(columns=drop_cols, inplace=True)

признак rank_cat

In [ ]:
print(f"Уникальных значений rank_cat: {df.rank_cat.nunique()}")
print(f"\n{df.rank_cat.unique()}") 

In [ ]:
# посмотрим на сколько они пересекаются с признаком sub_cat
df[['sub_cat', 'rank_cat']].sample(20)

In [ ]:
# Оставим только последние значения после символа > если он есть.
df.loc[~df.rank_cat.isnull(), 'rank_cat'] = df.loc[~df.rank_cat.isnull(), 'rank_cat'].apply(lambda s: s.split(' > ')[-1])

In [ ]:
# посмотрим на уникальные значения
df.rank_cat.unique()

In [ ]:
# заполним пропуски 
df.rank_cat.fillna(method='backfill', inplace=True)
print(f"Empty values {df.rank_cat.isnull().sum()}")

Признак rank_cat нуждается в очистке.  
Например от символов '].  
Так же стоит исключить разные написания одной и той же категории, например GroceryGourmetFood и Grocery & Gourmet Food и т.д.

In [ ]:
%%time
df.rank_cat = df.rank_cat.str.replace("']", "") \
                         .str.replace(' & ', '_') \
                         .str.replace(', ', '_') \
                         .str.replace(' ', '_') \
                         .str.lower()

In [ ]:
df.rank_cat.value_counts()

In [ ]:
%%time
# сохраним множество всех уникальных значений данного признака в переменную rank_val_set
rank_cat_set = set(df.rank_cat.unique())

# приведем категории к одному стилю написания с нижней чертой
for tmp_cat in rank_cat_set:
    if '_' in tmp_cat:
        #  get indexes feature values without "_"
        tmp_inds = df[df.rank_cat == tmp_cat.replace('_', '')].index
        if len(tmp_inds) != 0:
            # rank_col_index = data.columns.to_list().index('rank_cat')
            # change the value in rank_cat
            df.loc[tmp_inds, 'rank_cat'] = tmp_cat

In [ ]:
# так же заменим несколько значений на более читаемые
df.rank_cat = df.rank_cat.str.replace("healthhousehold", "health_household") \
                         .str.replace('sportsoutdoors', 'sports_outdoors') \
                         .str.replace('beautypersonalcare', 'beauty_personal_care') \
                         .str.replace('petsupplies', 'pet_supplies')

In [ ]:
#  посмотрим на гистрограмму rank_cat
sns.catplot(y="rank_cat", kind="count",
            palette="pastel", edgecolor=".6",
            data=df)

In [ ]:
df.rank_cat.value_counts(normalize=True)

Как видно по гистрограмме 94% всех наблюдений признака rank_cat имеют значение grocery_gourmet_food.  
Это никак нам не поможет, так как большая часть других значений, но своей сути являтеся подкатегорием топовой категории, например iced_tea, halva, olive и т.д.  
Принято решение удалить данный признак.

In [ ]:
df.drop(columns='rank_cat', inplace=True)

#### признак also_view  
also_view - идентификаторы товаров, которые так же смотрел пользователь  
Есть несколько задумок, которые попробую реализовать на этапе генерации новых признаков.  
Сейчас посмотрим сколько товаров обычно смотрят пользователи.

In [ ]:
#  На этапе объединения датасетов пропуски признака also_view были заменены на строку 'nan' 
# Сейчас вернем пропуски обратно и посмотрим 
df.also_view = df.also_view.apply(lambda s: np.nan if s == 'nan' else s)

In [ ]:
df.also_view.isnull().sum()

In [ ]:
# Создадим сначала булевый признак является ли значение реальным или искусственным
df['also_view_isNAN'] = pd.isna(df['also_view']).astype('uint8')

In [ ]:
# Добавим признак - кол-во просмотренных товаров, если был пропуск в also_view, то укажем 0
df['also_view_count'] = df['also_view'].apply(lambda s: len(s.split('|')) if not pd.isna(s) else 0)

In [ ]:
# Подсчитаем среднее кол-во просмотров товаров
mean = np.round(df['also_view_count'].mean())
print(f"В среднем пользователи смотрят {mean} товара.")

In [ ]:
diagnostic_plots_2(df, 'also_view_count', 'also_view_count - original')

Опытным путем установлено, что получить более нормальное распределение признака не представляется возможным.  
При этом на графике распределения можно выделить 3 группы значений:  
- от 0 до 10  
- от 11 до 40  
- от 41 и выше  
Таким образом решено привести данный признак к категориальному. Для этого добавим новый признак also_view_lvl

In [ ]:
df['also_view_lvl'] = df.also_view_count.apply(lambda n: 1 if n < 10 else 3 if n > 40 else 2)

In [ ]:
describe_cat_feature(df, 'also_view_lvl')

In [ ]:
drop_cols = ['also_view_count']
df.drop(columns=drop_cols, inplace=True)

#### признак also_buy
also_buy - перечень уникальных идентификаторов ранее купленных товаров

In [ ]:
# Сейчас вернем пропуски обратно и посмотрим 
df.also_buy = df.also_buy.apply(lambda s: np.nan if s == 'nan' else s)

In [ ]:
df.also_buy.isnull().sum()

In [ ]:
# Создадим сначала булевый признак является ли значение реальным или искусственным
df['also_buy_isNAN'] = pd.isna(df['also_buy']).astype('uint8')

In [ ]:
# Добавим признак - кол-во просмотренных товаров, если был пропуск в also_view, то укажем 0
df['also_buy_count'] = df['also_buy'].apply(lambda s: len(s.split('|')) if not pd.isna(s) else np.nan)

In [ ]:
# Подсчитаем среднее кол-во просмотров товаров
also_buy_mean = np.round(df['also_buy_count'].mean())
print(f"В среднем пользователи купили по {also_buy_mean} товаров.")

In [ ]:
# Заполним пропуски средним значением
df.also_buy_count.fillna(method='backfill', inplace=True)

In [ ]:
diagnostic_plots_2(df, 'also_buy_count', 'also_buy_count - original')

Попытки привести распределение признака also_buy_count к нормальнову виду не увенчались успехом. Во всех случаях наблюдается либо значительный хвост слева, либо появляется вторая мода.  
Были испробованы варианты:  
- логарифмирование  
- обратное преобразование
- возведение в степень 0.333, 0,5, 1.5, 1.8, 2, 2.5, 3.3  
- преобразование Бокса-Кокса  
- преобразование Йео-Джонсона  

Оставим оригинальное значение, остальные удалим.

#### признак price

In [ ]:
df.price.isnull().sum()

In [ ]:
# визуально оценим признак
df.price.sample(15)

In [ ]:
# Создадим сначала булевый признак является ли значение реальным или искусственным
df['price_isNAN'] = pd.isna(df['price']).astype('uint8')

In [ ]:
# Удалим знак $ перед значением
df.price = df.price.str.replace('$', '')
df.price = df.price.astype('object')

df.loc[~df.price.isnull(), 'price'] = df.loc[~df.price.isnull(), 'price'].apply(lambda s: np.round(np.mean(list(map(float, s.split(' - ')))), 2) if ' - ' in s else s)

# переведем к типу данных float
df.price = df.price.astype('float64')

In [ ]:
diagnostic_plots_2(df, 'price', 'price - original')

In [ ]:
# Подсчитаем среднее и медиану
price_mean = np.round(df['price'].mean())
price_median = np.round(df['price'].median())
print(f"Ср. цена товара ${price_mean}, медиана ${price_median}.")

Из графиков выше выдно, что признак имеет выбросы.  

Попробуем получить среднуюю цену и медиану стоимости товаров при группировке по категориям sub_cat

In [ ]:
agg_func_math = {
    'price': ['mean', 'median']
}
price_per_sub_cat = df.groupby(by='sub_cat').agg(agg_func_math).reset_index().round(2)
display(price_per_sub_cat)

Заполним пропуски в признаке price медианными значениями из датафрейма df_grouped с учетом категории, в которой находится товар

In [ ]:
%%time

for tmp_cat in price_per_sub_cat.sub_cat.values:
    # new features by price
    df.loc[(df.price.isnull()) & (df.sub_cat == tmp_cat), 'price'] = price_per_sub_cat[price_per_sub_cat.sub_cat == tmp_cat]['price']['median'].to_list()[0]

In [ ]:
diagnostic_plots_2(df, 'price', 'price - filled empty')

In [ ]:
plt.rcParams['figure.figsize'] = (20,8)
df['price'].hist(bins=150)

In [ ]:
# Логарифмическое преобразование  
df['price_log'] = np.log(df['price'])

# Преобразование Бокса-Кокса
df['price_boxcox'], param = stats.boxcox(df['price']) 
print('Оптимальное значение λ_price_boxcox = {}'.format(param))

# Преобразование Йео-Джонсона
df['price_yeojohnson'], param = stats.yeojohnson(df['price']) 
print('Оптимальное значение λ_price_yeojohnson = {}'.format(param))


In [ ]:
df['price_log'].hist(bins=40)

In [ ]:
df['price_yeojohnson'].hist(bins=40)

In [ ]:
df['price_boxcox'].hist(bins=40)

Признак price, после преобразования Йео-Джонсона имеет нормальное распределение.  
Будем использовать признак **price_yeojohnson**

In [ ]:
# удалим не нужные производные признаки
drop_cols = ['price_log', 'price_boxcox']
df.drop(columns=drop_cols, inplace=True)

Примерно 600 тыс. покупок, что составляет примерно 50% наблюдений составили товары со стоимостью 15-16 долларов, так же 380 тыс.ед. или 32% товаров купили по цене 20 долларов.

In [ ]:
gc.collect()

## Feature engineering  
На этом шаге мы еще раз пройдемся по признакам и создадим на их основе новые.

#### признаки reviewTime и unixReviewTime  
Создадим на основе признака reviewTime след. признаки:  
- reviewMonth (+)
- reviewDate (+)
- reviewYear (+)
- reviewWeek (+)
- reviewWeekday (+)
- day_of_year (+) - The ordinal day of the year.
- day_of_week (+)


На их основе создадим следующие признаки:  
- isSummer (+)
- isWinter (+)
- isAutumn (+)
- isSpring (+)
- isWeekends (+)

In [ ]:
# unixReviewTime - дата в unix формате

tsmin = datetime.utcfromtimestamp(df.unixReviewTime.min()).strftime('%Y-%m-%d %H:%M:%S')
tsmax = datetime.utcfromtimestamp(df.unixReviewTime.max()).strftime('%Y-%m-%d %H:%M:%S')
print(f"tsmin: {tsmin}")
print(f"tsmax: {tsmax}")

In [ ]:
#  высчитаем квантили по train_df
def get_main_quantiles(s):
    # check the type of s
    if type(df.unixReviewTime) != pd.Series:
        print(f"[Error in get_main_quantiles] the type data is wrong.")
        return
    
    return {
        'tsmin': s.min(),
        'ts25': int(s.quantile(0.25)),
        'ts50': int(s.quantile(0.50)),
        'ts75': int(s.quantile(0.75)),
        'tsmax': s.max(),
    }

time_quantiles = get_main_quantiles(df.unixReviewTime)

def cat_date(x, quant_dict):
    if x <= quant_dict['ts25']: x = 'old'
    elif quant_dict['ts25'] < x < quant_dict['ts75']: x = 'middle'
    elif quant_dict['ts75'] <= x: x = 'new'
    return x      

# Заменим значения в столбце на категории
df['unixReviewTime'] = df['unixReviewTime'].apply(lambda x: cat_date(x, time_quantiles))

In [ ]:
#  переведем признак в datetime формат
df['reviewTime'] = pd.to_datetime(df['reviewTime'], format='%m %d, %Y')

# Создание новых признаков на основе reviewTime
df['reviewMonth'] = df['reviewTime'].dt.month
df['reviewDate'] = df['reviewTime'].dt.day
df['reviewYear'] = df['reviewTime'].dt.year
df['reviewWeek'] = df['reviewTime'].dt.week
df['reviewWeekday'] = df['reviewTime'].dt.weekday # The day of the week with Monday=0, Sunday=6
df['day_of_year'] = df['reviewTime'].dt.dayofyear # The ordinal day of the year.
df['day_of_week'] = df['reviewTime'].dt.dayofweek
df['isSummer'] = df['reviewMonth'].apply(lambda x: x in [6, 7, 8]).astype('uint8')
df['isWinter'] = df['reviewMonth'].apply(lambda x: x in [12, 1, 2]).astype('uint8')
df['isAutumn'] = df['reviewMonth'].apply(lambda x: x in [9, 10, 11]).astype('uint8')
df['isSpring'] = df['reviewMonth'].apply(lambda x: x in [3, 4, 5]).astype('uint8')
df['isWeekends'] = df['day_of_week'].apply(lambda x: x in [5, 6]).astype('uint8') # выходные  

def determine_season(month):
    if month in [6, 7, 8]:
        return 'Summer'
    elif month in [12, 1, 2]:
        return 'Winter'
    elif month in [9, 10, 11]:
        return 'Autumn'
    return 'Spring'

df['season'] = df['reviewMonth'].apply(determine_season)

На основе новых признаков подсчитаем следующие значения:  
- кол-во покупок по месяцам  
- кол-во покупок по годам  
- кол-во покупок по неделям  
- кол-во покупок по временам года ( для этого создадим вр.признак season)
- кол-во покупок в будни  
- кол-во покупок по выходным  
- средняя цена покупок по месяцам  
- средняя цена покупок по годам  
- средняя цена покупок по неделям  
- средняя цена покупок по временам года
- средняя цена покупок в будни  
- средняя цена покупок по выходным

In [ ]:
# посмотрим на распределение в получившемся признаке season
describe_cat_feature(df, 'season')

Из графике можно сделать вывод, что в целом покупки совершались почти одинаково в течении всего года, однако в зимний и весенний периоды их совершено больше, чем летом.  
Посмотрим на данные по цене в зависимости от сезона года

In [ ]:
# сгруппируем данные по сезонам
agg_func_math = {
    'price': ['mean', 'count', 'sum']
}
season_grouped = df.groupby(by='season').agg(agg_func_math).round(2).reset_index()
display(season_grouped)

In [ ]:
plt.rcParams['figure.figsize']= 8, 6
sns.barplot(x="season", y=('price',  'sum'), data=season_grouped)

Добавим новый признак total_sum_by_season  

In [ ]:
%%time
# создадим пустой признак для последующего заполнения
df['num_reviews_by_season'] = np.nan

for tmp_season in df.season.unique():
    tmp_count = season_grouped[season_grouped.season == tmp_season]['price']['count'].to_list()[0]
    df.loc[df.season == tmp_season, 'num_reviews_by_season'] = tmp_count

In [ ]:
# # сгруппируем данные по reviewYear
agg_func_math = {
    'price': ['mean', 'count', 'sum']
}
year_grouped = df.groupby(by='reviewYear').agg(agg_func_math).round(2).reset_index()
display(year_grouped)

In [ ]:
plt.rcParams['figure.figsize']= 16, 6
sns.barplot(x="reviewYear", y=('price',  'sum'), data=year_grouped)

На этом барплоте хорошо видно, что максимальные суммы продаж были достигнуты в 2015 и 2016 годах

In [ ]:
# Построим график кол-ва обзоров по годам
plt.rcParams['figure.figsize']= 16, 6
sns.barplot(x="reviewYear", y=('price',  'count'), data=year_grouped)

Как видно, наибольшее кол-во обзоров на товары были в период с 2015 по 2017 года.  
Странно, что кол-во обзоров далее пошло на убывание, ведь, наверняка, кол-во пользователей онлайн магазинов только увеличивалось.  
Можно предположить, что в эти года активно использовали маркетинговые инструменты для стимулирования действия клиента по написанию отзыва о товаре.  
При этом до 2005 года отзывово почти нет. Это может говорить нам о том, что только к 2006 года компания увидела некоторые перспективы от внедрения отзывов.  
Кол-во отзывов по годам полностью коррелирует с общей суммой продаж по годам.  
Так как сумма продаж по годам не отражает объективно сумму продаж компании за этот период, так как мы имеем только данные по суммам товаров, на которые оставлены отзывы.  
Поэтому, добавим в основной датафрейм признак num_reviews_by_year

In [ ]:
%%time
# создадим пустой признак для последующего заполнения
df['num_reviews_by_year'] = np.nan

for tmp_year in df.reviewYear.unique():
    total_reviews = year_grouped[year_grouped.reviewYear == tmp_year]['price']['count'].to_list()[0]
    df.loc[df.reviewYear == tmp_year, 'num_reviews_by_year'] = total_reviews

In [ ]:
# посмотрим так же на признак reviewMonth
describe_cat_feature(df, 'reviewMonth')

Видно, что больше всего покупали с Декабря по Март включительно.  
Хуже всего покупали осенью

In [ ]:
# сгруппируем данные по месяцам
agg_func_math = {
    'price': ['mean', 'median', 'count']
}
month_grouped = df.groupby(by='reviewMonth').agg(agg_func_math).round(2).reset_index()
display(month_grouped)

Добавим признак num_reviews_by_month

In [ ]:
%%time
# создадим пустой признак для последующего заполнения
df['num_reviews_by_month'] = np.nan

for tmp in month_grouped.reviewMonth.unique():
    tmp_count = month_grouped[month_grouped.reviewMonth == tmp]['price']['count'].to_list()[0]
    df.loc[df.reviewMonth == tmp, 'num_reviews_by_month'] = tmp_count

In [ ]:
# сгруппируем данные по неделям
agg_func_math = {
    'price': ['mean', 'count']
}
week_grouped = df.groupby(by='reviewWeek').agg(agg_func_math).round(2).reset_index()
display(week_grouped.head(5))

In [ ]:
plt.rcParams['figure.figsize']= 20, 4
sns.barplot(x="reviewWeek", y=('price',  'count'), data=week_grouped)

In [ ]:
%%time
# создадим признак num_reviews_by_week
df['num_reviews_by_week'] = np.nan

for tmp in week_grouped.reviewWeek.unique():
    tmp_count = week_grouped[week_grouped.reviewWeek == tmp]['price']['count'].to_list()[0]
    df.loc[df.reviewWeek == tmp, 'num_reviews_by_week'] = tmp_count

* кол-во покупок в будни
* кол-во покупок по выходным
* средняя цена покупок по месяцам
* средняя цена покупок по годам
* средняя цена покупок по неделям
* средняя цена покупок по временам года
* средняя цена покупок в будни
* средняя цена покупок по выходным

In [ ]:
# кол-во покупок в будни и кол-во покупок по дням недели
# сгруппируем данные по неделям
agg_func_math = {
    'price': ['min', 'max', 'mean', 'median', 'std', 'count'],
    'also_buy_count': ['min', 'max', 'mean', 'median', 'std'],
    'rank_val_yeojohnson': ['min', 'max', 'mean', 'median', 'std'],
}
day_of_week_grouped = df.groupby(by='day_of_week').agg(agg_func_math).round(2).reset_index()
display(day_of_week_grouped)

In [ ]:
%%time
#  новые признаки
df['num_reviews_by_week_day'] = np.nan # -
df['mean_price_by_week_day'] = np.nan # -
df['std_price_by_week_day'] = np.nan # -
df['mean_also_buy_count_by_week_day'] = np.nan # -
df['std_also_buy_count_by_week_day'] = np.nan # -
df['mean_rank_val_yeojohnson_by_week_day'] = np.nan # -
df['std_rank_val_yeojohnson_by_week_day'] = np.nan # -


for tmp in day_of_week_grouped.day_of_week.unique():
    # new features by price
    df.loc[df.day_of_week == tmp, 'num_reviews_by_week_day'] = day_of_week_grouped[day_of_week_grouped.day_of_week == tmp]['price']['count'].to_list()[0]
    df.loc[df.day_of_week == tmp, 'mean_price_by_week_day'] = day_of_week_grouped[day_of_week_grouped.day_of_week == tmp]['price']['mean'].to_list()[0]
    df.loc[df.day_of_week == tmp, 'std_price_by_week_day'] = day_of_week_grouped[day_of_week_grouped.day_of_week == tmp]['price']['std'].to_list()[0]
    
    # new features by also_buy_count
    df.loc[df.day_of_week == tmp, 'mean_also_buy_count_by_week_day'] = day_of_week_grouped[day_of_week_grouped.day_of_week == tmp]['also_buy_count']['mean'].to_list()[0]
    df.loc[df.day_of_week == tmp, 'std_also_buy_count_by_week_day'] = day_of_week_grouped[day_of_week_grouped.day_of_week == tmp]['also_buy_count']['std'].to_list()[0]
    
    # new features by rank_val_yeojohnson
    df.loc[df.day_of_week == tmp, 'mean_rank_val_yeojohnson_by_week_day'] = day_of_week_grouped[day_of_week_grouped.day_of_week == tmp]['rank_val_yeojohnson']['mean'].to_list()[0]
    df.loc[df.day_of_week == tmp, 'std_rank_val_yeojohnson_by_week_day'] = day_of_week_grouped[day_of_week_grouped.day_of_week == tmp]['rank_val_yeojohnson']['std'].to_list()[0]

In [ ]:
# сгруппируем данные по номеру дня в году
agg_func_math = {
    'price': ['min', 'max', 'mean', 'median', 'std', 'count'],
    'also_buy_count': ['min', 'max', 'mean', 'median', 'std'],
    'rank_val_yeojohnson': ['min', 'max', 'mean', 'median', 'std'],
}
day_of_year_grouped = df.groupby(by='day_of_year').agg(agg_func_math).round(2).reset_index()
display(day_of_year_grouped)

In [ ]:
%%time
# by price => mean, std, count
df['num_reviews_by_year_day'] = np.nan # -
df['mean_price_by_year_day'] = np.nan # -
df['std_price_by_year_day'] = np.nan # -
# by also_buy_count => mean, median, std
df['mean_also_buy_count_by_year_day'] = np.nan # -
df['median_also_buy_count_by_year_day'] = np.nan # -
df['std_also_buy_count_by_year_day'] = np.nan # -
# by rank_val_yeojohnson => min, max, mean, median, std
df['min_rank_val_yeojohnson_by_year_day'] = np.nan # -
df['max_rank_val_yeojohnson_by_year_day'] = np.nan # -
df['mean_rank_val_yeojohnson_by_year_day'] = np.nan # -
df['median_rank_val_yeojohnson_by_year_day'] = np.nan # -
df['std_rank_val_yeojohnson_by_year_day'] = np.nan # -

for tmp in day_of_year_grouped.day_of_year.unique():
    # new features by price
    df.loc[df.day_of_year == tmp, 'num_reviews_by_year_day'] = day_of_year_grouped[day_of_year_grouped.day_of_year == tmp]['price']['count'].to_list()[0]
    df.loc[df.day_of_year == tmp, 'mean_price_by_year_day'] = day_of_year_grouped[day_of_year_grouped.day_of_year == tmp]['price']['mean'].to_list()[0]
    df.loc[df.day_of_year == tmp, 'std_price_by_year_day'] = day_of_year_grouped[day_of_year_grouped.day_of_year == tmp]['price']['std'].to_list()[0]
    
    # new features by also_buy_count
    df.loc[df.day_of_year == tmp, 'mean_also_buy_count_by_year_day'] = day_of_year_grouped[day_of_year_grouped.day_of_year == tmp]['also_buy_count']['mean'].to_list()[0]
    df.loc[df.day_of_year == tmp, 'median_also_buy_count_by_year_day'] = day_of_year_grouped[day_of_year_grouped.day_of_year == tmp]['also_buy_count']['median'].to_list()[0]
    df.loc[df.day_of_year == tmp, 'std_also_buy_count_by_year_day'] = day_of_year_grouped[day_of_year_grouped.day_of_year == tmp]['also_buy_count']['std'].to_list()[0]
    
#     new features by rank_val_yeojohnson
    df.loc[df.day_of_year == tmp, 'min_rank_val_yeojohnson_by_year_day'] = day_of_year_grouped[day_of_year_grouped.day_of_year == tmp]['rank_val_yeojohnson']['min'].to_list()[0]
    df.loc[df.day_of_year == tmp, 'max_rank_val_yeojohnson_by_year_day'] = day_of_year_grouped[day_of_year_grouped.day_of_year == tmp]['rank_val_yeojohnson']['max'].to_list()[0]
    df.loc[df.day_of_year == tmp, 'mean_rank_val_yeojohnson_by_year_day'] = day_of_year_grouped[day_of_year_grouped.day_of_year == tmp]['rank_val_yeojohnson']['mean'].to_list()[0]
    df.loc[df.day_of_year == tmp, 'median_rank_val_yeojohnson_by_year_day'] = day_of_year_grouped[day_of_year_grouped.day_of_year == tmp]['rank_val_yeojohnson']['median'].to_list()[0]
    df.loc[df.day_of_year == tmp, 'std_rank_val_yeojohnson_by_year_day'] = day_of_year_grouped[day_of_year_grouped.day_of_year == tmp]['rank_val_yeojohnson']['std'].to_list()[0]

Посмотрим какие новые признаки мы сможем получить из признака vote

In [ ]:
# сгруппируем данные по кол-ву голосов за отзыв
agg_func_math = {
    'price': ['min', 'max', 'mean', 'median', 'std', 'count'],
    'also_buy_count': ['min', 'max', 'mean', 'median', 'std']
}
vote_grouped = df.groupby(by='vote').agg(agg_func_math).round(2).reset_index()
display(vote_grouped)

In [ ]:
%%time
#  новые признаки
df['num_reviews_by_vote'] = np.nan # +
df['mean_price_by_vote'] = np.nan # +
# df['median_price_by_vote'] = np.nan # + признак имеет только 1 значение - удалить
df['std_price_by_vote'] = np.nan # +
df['mean_also_buy_count_by_vote'] = np.nan # +
df['median_also_buy_count_by_vote'] = np.nan # +
df['std_also_buy_count_by_vote'] = np.nan # +


for tmp in vote_grouped.vote.unique():
    # new features by price
    df.loc[df.vote == tmp, 'num_reviews_by_vote'] = vote_grouped[vote_grouped.vote == tmp]['price']['count'].to_list()[0]
    df.loc[df.vote == tmp, 'mean_price_by_vote'] = vote_grouped[vote_grouped.vote == tmp]['price']['mean'].to_list()[0]
#     df.loc[df.vote == tmp, 'median_price_by_vote'] = vote_grouped[vote_grouped.vote == tmp]['price']['median'].to_list()[0] # + признак имеет только 1 значение - удалить
    df.loc[df.vote == tmp, 'std_price_by_vote'] = vote_grouped[vote_grouped.vote == tmp]['price']['std'].to_list()[0]
    
    # new features by also_buy_count
    df.loc[df.vote == tmp, 'mean_also_buy_count_by_vote'] = vote_grouped[vote_grouped.vote == tmp]['also_buy_count']['mean'].to_list()[0]
    df.loc[df.vote == tmp, 'median_also_buy_count_by_vote'] = vote_grouped[vote_grouped.vote == tmp]['also_buy_count']['median'].to_list()[0]
    df.loc[df.vote == tmp, 'std_also_buy_count_by_vote'] = vote_grouped[vote_grouped.vote == tmp]['also_buy_count']['std'].to_list()[0]

itemid  
Сгруппируем данные по признаку itemid и узнаем сколько отзывов оставлено о каждом товаре.  
Добавим полученную информацию в качестве признака itemid_reviews_count  
Так же, используя признак itemid, получим следующие признаки  min, max, mean, median, std

In [ ]:
# сгруппируем данные по кол-ву голосов за отзыв
agg_func_math = {
    'overall': ['min', 'max', 'mean', 'median', 'std', 'count']
}
overall_per_itemid = df[df.is_train == 1].groupby(by='itemid').agg(agg_func_math).round(3).reset_index()
display(overall_per_itemid)

In [ ]:
%%time
#  новые признаки
df['reviews_per_itemid'] = np.nan
df['min_overall_by_itemid'] = np.nan # +
df['max_overall_by_itemid'] = np.nan # +
df['mean_overall_by_itemid'] = np.nan # +
df['median_overall_by_itemid'] = np.nan # +
df['std_overall_by_itemid'] = np.nan # +

for tmp in overall_per_itemid.itemid:
    # new features by price
    df.loc[df.itemid == tmp, 'reviews_per_itemid'] = overall_per_itemid[overall_per_itemid.itemid == tmp]['overall']['count'].values[0]
    df.loc[df.itemid == tmp, 'min_overall_by_itemid'] = overall_per_itemid[overall_per_itemid.itemid == tmp]['overall']['min'].values[0]
    df.loc[df.itemid == tmp, 'max_overall_by_itemid'] = overall_per_itemid[overall_per_itemid.itemid == tmp]['overall']['max'].values[0]
    df.loc[df.itemid == tmp, 'mean_overall_by_itemid'] = overall_per_itemid[overall_per_itemid.itemid == tmp]['overall']['mean'].values[0]
    df.loc[df.itemid == tmp, 'median_overall_by_itemid'] = overall_per_itemid[overall_per_itemid.itemid == tmp]['overall']['median'].values[0]
    df.loc[df.itemid == tmp, 'std_overall_by_itemid'] = overall_per_itemid[overall_per_itemid.itemid == tmp]['overall']['std'].values[0]


In [ ]:
# посмотирм на уникальные значения получившегося признака max_overall_by_itemid
df.max_overall_by_itemid.unique()

#### Признак i_desc  
Это текстовый признак, полученный в результаты объединения признака title и description, которые описывают товар.  


In [ ]:
# # запустить при подключенном интернет соединеии
# nltk.download('punkt') # At first you have to download these nltk packages.
# nltk.download('stopwords')
# nltk.download('wordnet')

In [ ]:
stop_words = stopwords.words('english') # defining stop_words
stop_words.remove('not')
lemmatizer = WordNetLemmatizer()

def data_preprocessing(string):
    
    # data cleaning
    string = re.sub(re.compile('<.*?>'), '', string) #removing html tags
    string =  re.sub('[^A-Za-z0-9]+', ' ', string) #taking only words
    string = re.sub(r"www.+\s+?", "", string) # removing links starts with www. 
  
    # lowercase
    string = string.lower()
  
    # tokenization
    tokens = nltk.word_tokenize(string) # converts review to tokens
  
    # stop_words removal
    string = [word for word in tokens if word not in stop_words] #removing stop words
  
    # lemmatization
    string = [lemmatizer.lemmatize(word) for word in string]
  
    # join words in preprocessed review
    string = ' '.join(string)
  
    return string

- обработаем текстовый признак с помощью функции data_preprocessing  
- TFIDF векторизация  
- уменьшение размерности полученного вектора с помощью TruncatedSVD

In [ ]:
%%time
# text preprocessing
df['i_desc'] = df['i_desc'].apply(data_preprocessing)
print('data_preprocessing completed')

# garbage collect
gc.collect()

In [ ]:
%%time
# TFIDF векторизация
vectorizer = TfidfVectorizer(min_df=20, max_features=200)
X_tfidf = vectorizer.fit_transform(df['i_desc'])
print('X_tfidf shape: ', X_tfidf.shape)

# Dimensionality reduction
NUM_COMPONENTS = 20
svd = TruncatedSVD(n_components=NUM_COMPONENTS, n_iter=7, random_state=42)
X_reduced = svd.fit_transform(X_tfidf)
reduced_columns = [f"i_desc_{i+1}" for i in range(NUM_COMPONENTS)]

# create a dataFrame
X_reduced_df = pd.DataFrame(data=X_reduced, columns=reduced_columns)
print(X_reduced_df.shape)

# concatenate df and X_reduced_df
frames = [df.reset_index(), X_reduced_df]
df = pd.concat(frames, axis=1).set_index(keys='index')

print(f"df shape: {df.shape}")
display(df.head(3))

# garbage collect
gc.collect()

#### Признаки reviewText и summary  
Это отзывы пользователей на конкретный товар.  
В теории из данных признаков можно получить очень ценные для рекомендательной системы признаки.

In [ ]:
# concatenate reviewText and summary
df['revsum'] = df.reviewText + ' ' + df.summary

# fill na 
df['revsum'].fillna(value='noData', inplace=True)

# drop initial features
df.drop(columns=['reviewText', 'summary'], inplace=True)

# collect garbage
gc.collect()

In [ ]:
%%time
# text preprocessing
df['revsum'] = df['revsum'].apply(data_preprocessing)
print('data_preprocessing completed')

# garbage collect
gc.collect()

In [ ]:
# TFIDF векторизация
vectorizer = TfidfVectorizer(min_df=20, max_features=200)
X_tfidf = vectorizer.fit_transform(df['revsum'])
print('X_tfidf shape: ', X_tfidf.shape)

# Dimensionality reduction
NUM_COMPONENTS = 20
svd = TruncatedSVD(n_components=NUM_COMPONENTS, n_iter=7, random_state=42)
X_reduced = svd.fit_transform(X_tfidf)
reduced_columns = [f"revsum_{i+1}" for i in range(NUM_COMPONENTS)]

# create a dataFrame
X_reduced_df = pd.DataFrame(data=X_reduced, columns=reduced_columns)
print(X_reduced_df.shape)

# concatenate df and X_reduced_df
frames = [df, X_reduced_df]
df = pd.concat(frames, axis=1)

print(f"df shape: {df.shape}")
display(df.head(3))

# garbage collect
gc.collect()

In [ ]:
# удалим признаки i_desc и revsum
df.drop(columns=['i_desc', 'revsum'], inplace=True)
# garbage collect
gc.collect()

Для уменьшения веса набора данных, изменим типы некоторых данных

In [ ]:
df.info(max_cols=160, memory_usage=True)

In [ ]:
# удалим признаки also_view и also_buy для экономии памяти
df.drop(columns=['also_view', 'also_buy'], inplace=True)

Сохраним и выгрузим подготовленный датафрейм в csv файл для ускорения последующей работы

In [ ]:
# Для оптимизации памяти, нужно записать полученный на такущем этапе датафрейм в csv формат, рестартануть кернел и получить данные
df.to_csv('prepared_data.csv', index=False)

In [2]:
#  чтение данных после перезагрузки кернела

col_types = {'verified': 'uint8',
             'is_train': 'uint8',
             'vote_isNAN': 'uint8',
             'desc_isNAN': 'uint8',
             'title_isNAN': 'uint8',
             'rank_isNAN': 'uint8',
             'also_view_isNAN': 'uint8',
             'also_view_lvl': 'uint8',
             'also_buy_isNAN': 'uint8',
             'price_isNAN': 'uint8',
}

df = pd.read_csv('/kaggle/input/prepared-data-09-05/prepared_data_09_05.csv', dtype=col_types, index_col=0, low_memory=False)
df.shape

In [ ]:
df.info(max_cols=160, memory_usage=True)

#### one-hot кодирование

In [5]:
cols_to_ohe = ['unixReviewTime', 'vote', 'brand', 'sub_cat', 'also_view_lvl']
for col in cols_to_ohe:
    print(f"col {col}, unique: {df[col].nunique()}")

In [6]:
#  преобразуем признак unixReviewTime в категориальный
#  высчитаем квантили по train_df
def get_main_quantiles(s):
    # check the type of s
    if type(df.unixReviewTime) != pd.Series:
        print(f"[Error in get_main_quantiles] the type data is wrong.")
        return
    
    return {
        'tsmin': s.min(),
        'ts25': int(s.quantile(0.25)),
        'ts50': int(s.quantile(0.50)),
        'ts75': int(s.quantile(0.75)),
        'tsmax': s.max(),
    }

time_quantiles = get_main_quantiles(df.unixReviewTime)

def cat_date(x, quant_dict):
    if x <= quant_dict['ts25']: x = 'old'
    elif quant_dict['ts25'] < x < quant_dict['ts75']: x = 'middle'
    elif quant_dict['ts75'] <= x: x = 'new'
    return x      

# Заменим значения в столбце на категории
df['unixReviewTime'] = df['unixReviewTime'].apply(lambda x: cat_date(x, time_quantiles))

In [9]:
del time_quantiles
gc.collect()

In [8]:
cols_to_ohe = ['unixReviewTime', 'vote', 'brand', 'sub_cat', 'also_view_lvl']
prefix_names = {
    'unixReviewTime': 'reviewTime',
    'vote': 'vote',
    'brand': 'brand',
    'sub_cat': 'sub_cat',
    'also_view_lvl': 'view_lvl',
}

df = pd.get_dummies(df, prefix=prefix_names, columns=cols_to_ohe)
print(f"df shape: {df.shape}")

# После ohe-кодирования, в названиях признаков возникли значения, содержащие пробелы между словами.  
# Исправим этот момент.
fixed_column_names = ['_'.join(col.split(' ')) if ' ' in col else col for col in df.columns]
df.columns = fixed_column_names

### Устранение пропусков в данных

Для начала разделим данные на train и test

In [23]:
train = df.loc[df.is_train == 1, :].copy().drop(columns=['is_train', 'Id'])
test = df.loc[df.is_train == 0, :].copy().drop(columns=['overall', 'rating', 'is_train'])

In [12]:
# пропуски в train
train.isnull().sum()[train.isnull().sum() > 0]

In [13]:
# заменим пропуски медианным значением
train.std_overall_by_itemid = train.std_overall_by_itemid.fillna(value=df.std_overall_by_itemid.median())

In [14]:
# пропуски в test
test.isnull().sum()[test.isnull().sum() > 0]

In [15]:
# заменим пропуски медианным значением
test_empty_cols = 'min_overall_by_itemid max_overall_by_itemid mean_overall_by_itemid median_overall_by_itemid std_overall_by_itemid reviews_per_itemid'.split(' ')
for col_name in test_empty_cols:
    test[col_name] = test[col_name].fillna(value=df[col_name].median())

In [16]:
# поменяем типы данных некоторых признаков для сохранения доступной памяти
train['overall'] = train['overall'].astype('uint8')
train['rating'] = train['rating'].astype('uint8')
train['min_overall_by_itemid'] = train['min_overall_by_itemid'].astype('uint8')
train['max_overall_by_itemid'] = train['max_overall_by_itemid'].astype('uint8')

test['Id'] = test['Id'].astype('int32')
test['min_overall_by_itemid'] = test['min_overall_by_itemid'].astype('uint8')
test['max_overall_by_itemid'] = test['max_overall_by_itemid'].astype('uint8')

### Обработка выбросов в данных  
В первый раз обучим модель без удаления пропусков  
Затем сделаем тоже самое, но с удаленными/замененными пропусками

In [18]:
# несколко полезных функций для обработки выбросов
def get_stat_metrics(s):
    """
    Функция получает объект типа pandas.Series
    Расчитывает основные статистики:
    median - медиана
    perc25, perc75 - нижний и верхний квартили
    IQR - межквартильный размах
    bottom_border - нижняя граница
    upper_border - верхняя граница
    Возвращает словарь статистических значений.
    """
    q1 = s.quantile(0.25)
    q2 = s.median()
    q3 = s.quantile(0.75)
    iqr = q3 - q1
    fence_low = q1 - 1.5 * iqr
    fence_high = q3 + 1.5 * iqr
    minimum = s.min()
    maximum = s.max()
    
    names = 'q1 q2 q3 iqr fence_low fence_high minimum maximum'.split(' ')
    
    stats = dict(zip(names,
                     [q1, q2, q3, iqr, fence_low, \
                      fence_high, minimum, maximum]))
    
    return stats

Отберем числовые признаки в которых стоит проверить наличие выбросов

In [19]:
# отберем признаки, в которых присутсвуют выбросы
outliers = []

def get_outliers_features(s, feature_name):
    
    sd = get_stat_metrics(s)
    if (sd['minimum']<sd['fence_low']) or (sd['maximum']>sd['fence_high']):
        outliers.append(feature_name)
        

num_features = train.select_dtypes(include=['float64', 'int64', 'int32', 'int16']).columns

for col in num_features:
    get_outliers_features(df[col], col)

In [ ]:
outliers

Заменим выбросы на крайние значения границы выбросов

In [21]:
for data in [train, test]:
    for col in outliers:
        sd = get_stat_metrics(data[col])
        fence_low = sd['fence_low']  # нижняя граница выбросов
        fence_high = sd['fence_high'] # нижняя граница выбросов
        data.loc[data[col] > fence_high, col] = fence_high
        data.loc[data[col] < fence_low, col] = fence_low

## MODEL

In [24]:
train_data, test_data = train_test_split(train,random_state=32, shuffle=True)

In [25]:
ratings_coo = sparse.coo_matrix((train_data['rating'].astype(int),
                                 (train_data['userid'],
                                  train_data['itemid'])))

In [132]:
# использование item_features не дало прироста метрики
# укажем в item_features признаки,описывающие товар и затем сделаем разреженную матрицу item_features для дальнейшего использования в моделе
# item_features = train_data[item_cols]
# norm_ifeatures = (item_features - item_features.mean()) / item_features.std()
# item_features=(sparse.csr_matrix(norm_ifeatures)).astype(np.float32)

In [80]:
# использование user_features не дало прироста метрики
# сделаем разреженную матрицу user_features,содержащий признаки,описывающие потребителя
# user_features = train_data[user_cols]
# norm_ufeatures = (user_features - user_features.mean()) / user_features.std()
# user_features = (sparse.csr_matrix(norm_ufeatures)).astype(np.float32)

In [26]:
%%time
LR = 0.089
NUM_THREADS = 8 #число потоков
NUM_COMPONENTS = 160 #число параметров вектора 
NUM_EPOCHS = 25 #число эпох обучения
LEARNING_SCHEDULE = 'adagrad'
LOSS_FUNCTION = 'logistic'
RANDOM_STATE = 20

model = LightFM(learning_rate=LR,
                loss=LOSS_FUNCTION,
                no_components=NUM_COMPONENTS,
                learning_schedule = LEARNING_SCHEDULE,
                random_state = RANDOM_STATE
)
model = model.fit(ratings_coo,
                  epochs=NUM_EPOCHS,
                  num_threads=NUM_THREADS
)

In [30]:
#для предсказаний используем матрицы 
userid = np.array(test_data.userid).astype(np.int32)
itemid =np.array(test_data.itemid).astype(np.int32)

# Получаем предсказание:
preds0 = model.predict(userid, itemid, num_threads=NUM_THREADS)

# посчитаем метрику качества модели
sklearn.metrics.roc_auc_score(test_data.rating, preds0)

Обучим модель на всей тренировочной выборке и сделаем предсказание для сабмита

In [ ]:
ratings_coo_1 = sparse.coo_matrix((train['rating'].astype(int),
                                 (train['userid'],
                                  train['itemid'])))

In [41]:
%%time
LR = 0.089
NUM_THREADS = 8 #число потоков
NUM_COMPONENTS = 160 #число параметров вектора 
NUM_EPOCHS = 25 #число эпох обучения
LEARNING_SCHEDULE = 'adagrad'
LOSS_FUNCTION = 'logistic'
RANDOM_STATE = 20

model_1 = LightFM(learning_rate=LR,
                loss=LOSS_FUNCTION,
                no_components=NUM_COMPONENTS,
                learning_schedule = LEARNING_SCHEDULE,
                random_state = RANDOM_STATE
)
model_1 = model_1.fit(ratings_coo_1,
                  epochs=NUM_EPOCHS,
                  num_threads=NUM_THREADS
)


In [42]:
# для предсказаний используем матрицы 
userid_test = np.array(test.userid).astype(np.int32)
itemid_test =np.array(test.itemid).astype(np.int32)

# Получаем предсказание:
preds_1 = model_1.predict(userid_test, itemid_test, num_threads=NUM_THREADS)

In [43]:
print('Unnormalized preds: ', preds_1.min(), preds_1.max())
normalized_preds_1 = (preds_1 - preds_1.min())/(preds_1 - preds_1.min()).max()
print('Normalized preds', normalized_preds_1.min(), normalized_preds_1.max())

Сделаем сабмит

In [44]:
submission['rating']= normalized_preds_1
submission.to_csv('submission_log_10_05_ver_7.csv', index=False)


Опытным путем, установленно, что введение в модель дополнительных признаков, которые были мною обработаны, преобразованы, а так же их производные, никак не улучшают метрику качества модели, а наоборот приводят к худшим результатам.  
Остается открытым вопрос в работе с доп. признаками и их отбором. 
Я возлагал большие надежды на признаки, полученные из текстовых данных, однако они так же ухудшают метрику.  
На текущий момент самое лучшее достигнутое мною решение, ничем не отличается от базового.  
На этом решено прекратить попытки улучшения метрики, так как дедлайн подошел к концу.  
В целях отработки навыков работы с рекомендательными системами, ниже ниже приведу пример получения конкретных рекомендаций.

In [45]:
# Достаём эбмеддинги
item_biases, item_embeddings = model_1.get_item_representations()

In [46]:
item_biases.shape, item_embeddings.shape

In [ ]:
!pip install nmslib

In [49]:
import nmslib
 
#Создаём наш граф для поиска
nms_idx = nmslib.init(method='hnsw', space='cosinesimil')
 
#Начинаем добавлять наши товары в граф
nms_idx.addDataPointBatch(item_embeddings)
nms_idx.createIndex(print_progress=True)

In [57]:
#Вспомогательная функция для поиска по графу
def nearest_item_nms(itemid, index, n=10):
    nn = index.knnQuery(item_embeddings[itemid], k=n)
    return nn

In [52]:
mapper = dict(zip(meta['asin'],meta['title']))

In [53]:
# Создадим dataframe prod_id
train['title'] = train.asin.apply(lambda x: mapper[x])
test['title'] = test.asin.apply(lambda x: mapper[x])
prod_id = train.drop(['verified','reviewTime','reviewerName','reviewText','summary','unixReviewTime','vote','style','image'],axis=1)

In [54]:
prod_id.head(3)

Попробуем написать рекомендации к какому-нибудь товару. Например, к 'chocolate'.

In [55]:
prod_id[prod_id.title.str.find('chocolate')>=0].head(5)

возьмем item_id == 26561

In [58]:
# Ищем похожие товары
nbm = nearest_item_nms(26561, nms_idx)[0]

In [64]:
#Выведим их
prod_id[prod_id.itemid.isin(nbm)][['itemid', 'title']].drop_duplicates().head(10)

## Основные итоги работы.
В этом проекте повторены навыки EDA, data preprocessing, feature engineering и outliers processing. Так же на практике опробована простая рекомендательная система.  
Результат на лидерборде 38 место, score=0.76325. Результатом не доволен, однако, на текущий момент, имеющихся знаний не хватает, чтобы подняться по лидерборду выше.  
Опыт коммандной работы не отработан. Над проектом работал один.  